In [1]:
import polars as pl
import matplotlib.pyplot as plt
import os
import re

In [2]:
!pwd

/teamspace/studios/this_studio


In [3]:
catalog: pl.DataFrame = pl.read_csv(
    "nasa_space_apps_2024_seismic_detection_data/data/lunar/training/catalogs/apollo12_catalog_GradeA_final.csv"
)
catalog

filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid,mq_type
str,str,f64,str,str
"""xa.s12.00.mhz.1970-01-19HR00_e…","""1970-01-19T20:25:00.000000""",73500.0,"""evid00002""","""impact_mq"""
"""xa.s12.00.mhz.1970-03-25HR00_e…","""1970-03-25T03:32:00.000000""",12720.0,"""evid00003""","""impact_mq"""
"""xa.s12.00.mhz.1970-03-26HR00_e…","""1970-03-26T20:17:00.000000""",73020.0,"""evid00004""","""impact_mq"""
"""xa.s12.00.mhz.1970-04-25HR00_e…","""1970-04-25T01:14:00.000000""",4440.0,"""evid00006""","""impact_mq"""
"""xa.s12.00.mhz.1970-04-26HR00_e…","""1970-04-26T14:29:00.000000""",52140.0,"""evid00007""","""deep_mq"""
…,…,…,…,…
"""xa.s12.00.mhz.1974-10-14HR00_e…","""1974-10-14T17:43:00.000000""",63780.0,"""evid00156""","""impact_mq"""
"""xa.s12.00.mhz.1975-04-12HR00_e…","""1975-04-12T18:15:00.000000""",65700.0,"""evid00191""","""impact_mq"""
"""xa.s12.00.mhz.1975-05-04HR00_e…","""1975-05-04T10:05:00.000000""",36300.0,"""evid00192""","""impact_mq"""


In [4]:
catalog["time_rel(sec)"][0]

73500.0

# Lunar


## Training


In [5]:
training_dir = (
    "nasa_space_apps_2024_seismic_detection_data/data/lunar/training/data/S12_GradeA"
)

In [45]:
for i in range(len(catalog)):
    try:
        eventID = re.search("evid(\d+)", catalog["filename"][i]).group(1)
        data = pl.read_csv(f"{training_dir}/{catalog['filename'][i]}.csv")
        velocity_data = data["velocity(m/s)"]
        time_rel = data["time_rel(sec)"]
        mq_time_rel = catalog["time_rel(sec)"][i]
        MQ_TIME = data.with_columns(
            pl.when(pl.col("time_rel(sec)") >= mq_time_rel).then(1).otherwise(0)
        )["literal"]

        dictionary = {
            "mq": MQ_TIME.to_list(),
            "time_rel": time_rel.to_list(),
            "velocity": velocity_data.to_list(),
        }
        df = pl.DataFrame(dictionary)
        df.write_csv(f"seismic_data/data/train/moon/lunar_train_12_a_{eventID}.csv")
    except Exception as e:
        print(catalog["filename"][i] + " failed " + str(object=e))

xa.s12.00.mhz.1971-04-13HR00_evid00029 failed No such file or directory (os error 2): ...tion_data/data/lunar/training/data/S12_GradeA/xa.s12.00.mhz.1971-04-13HR00_evid00029.csv


## Testing


In [19]:
testing_dir = "nasa_space_apps_2024_seismic_detection_data/data/lunar/test/data/"
testing_dirs = [testing_dir + d for d in os.listdir(testing_dir)]

In [20]:
testing_dirs

['nasa_space_apps_2024_seismic_detection_data/data/lunar/test/data/S12_GradeB',
 'nasa_space_apps_2024_seismic_detection_data/data/lunar/test/data/S15_GradeA',
 'nasa_space_apps_2024_seismic_detection_data/data/lunar/test/data/S15_GradeB',
 'nasa_space_apps_2024_seismic_detection_data/data/lunar/test/data/S16_GradeA',
 'nasa_space_apps_2024_seismic_detection_data/data/lunar/test/data/S16_GradeB']

In [21]:
for j in range(len(testing_dirs)):
    for i in range(len(os.listdir(testing_dirs[j]))):
        try:
            filename = f"{testing_dirs[j]}/{os.listdir(testing_dirs[j])[i]}"
            s = re.search("s(\d+)", filename).group(1)
            grade = re.search("Grade([A-Z])", filename).group(1).lower()
            eventID = re.search("evid(\d+)", filename).group(1)
            if filename.endswith(".csv") is not True:
                continue
            # print(filename)
            # print(f"{s} {grade}")
            # print(eventID)
            velocity_data = pd.read_csv(filepath_or_buffer=filename)[
                "velocity(m/s)"
            ].to_list()
            time_rel = pd.read_csv(filepath_or_buffer=filename)[
                "time_rel(sec)"
            ].to_list()
            dictionary = {
                "time_rel": time_rel,
                "velocity": velocity_data,
            }
            df = pd.DataFrame(dictionary)
            df.to_csv(
                path_or_buf=f"data/lunar_test_{s}_{grade}_{eventID}.csv", index=False
            )
        except Exception as e:
            print(filename + " failed " + str(object=e))

# mars


## Training


In [22]:
training_dir = "nasa_space_apps_2024_seismic_detection_data/data/mars/training/data"
catalog = pd.read_csv(
    "nasa_space_apps_2024_seismic_detection_data/data/mars/training/catalogs/Mars_InSight_training_catalog_final.csv"
)
catalog

,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid
0,XB.ELYSE.02.BHV.2022-02-03HR08_evid0005.csv,2022-02-03T08:08:27.000000,507.0,evid0005
1,XB.ELYSE.02.BHV.2022-01-02HR04_evid0006.csv,2022-01-02T04:35:30.000000,2130.0,evid0006


In [23]:
for i in range(len(catalog)):
    try:
        eventID = re.search("evid(\d+)", catalog["filename"][i]).group(1)
        velocity_data = pd.read_csv(
            filepath_or_buffer=f"{training_dir}/{catalog['filename'][i]}"
        )["velocity(c/s)"].to_list()
        dictionary = {
            "time_rel": catalog["time_rel(sec)"][i],
            "velocity": velocity_data,
        }
        df = pd.DataFrame(dictionary)
        df.to_csv(path_or_buf=f"data/mars_train_{eventID}.csv", index=False)
    except Exception as e:
        print(catalog["filename"][i] + " failed " + str(object=e))

## Testing


In [24]:
testing_dirs = ["nasa_space_apps_2024_seismic_detection_data/data/mars/test/data"]

In [25]:
testing_dirs

['nasa_space_apps_2024_seismic_detection_data/data/mars/test/data']

In [26]:
for j in range(len(testing_dirs)):
    for i in range(len(os.listdir(testing_dirs[j]))):
        try:
            filename = f"{testing_dirs[j]}/{os.listdir(testing_dirs[j])[i]}"
            eventID = re.search("evid(\d+)", filename).group(1)
            if filename.endswith(".csv") is not True:
                continue
            # print(filename)
            # print(eventID)
            velocity_data = pd.read_csv(filepath_or_buffer=filename)[
                "velocity(c/s)"
            ].to_list()
            time_rel = pd.read_csv(filepath_or_buffer=filename)[
                "rel_time(sec)"
            ].to_list()
            dictionary = {
                "time_rel": time_rel,
                "velocity": velocity_data,
            }
            df = pd.DataFrame(dictionary)
            df.to_csv(path_or_buf=f"data/mars_test_{eventID}.csv", index=False)
        except Exception as e:
            print(filename + " failed " + str(object=e))